## Using built-in_Kmean

In [29]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

original_image = cv2.imread('Q1 Dataset\lady.png')
aux_image = cv2.imread('Q1 Dataset\lady stroke 1.png')

# Convert auxiliary image to HSV color space for more reliable color detection
aux_hsv = cv2.cvtColor(aux_image, cv2.COLOR_BGR2HSV)

# Adaptive thresholding for red and blue colors
red_lower = np.array([0, 100, 50])
red_upper = np.array([10, 255, 255])
blue_lower = np.array([100, 100, 50])
blue_upper = np.array([130, 255, 255])

# Create masks for red and blue
foreground_mask = cv2.inRange(aux_hsv, red_lower, red_upper)
background_mask = cv2.inRange(aux_hsv, blue_lower, blue_upper)

# Morphological operations for mask refinement
kernel = np.ones((5, 5), np.uint8)
foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_CLOSE, kernel)
background_mask = cv2.morphologyEx(background_mask, cv2.MORPH_CLOSE, kernel)

# Use masks to extract seed pixels from the original image
foreground_pixels = original_image[foreground_mask == 255]
background_pixels = original_image[background_mask == 255]

# Check if any seeds are detected
if foreground_pixels.size == 0 or background_pixels.size == 0:
    raise ValueError("No seeds detected. Please check the color thresholds and auxiliary image.")

# K-Means clustering on the foreground and background seed pixels
kmeans_fg = KMeans(n_clusters=8, random_state=42).fit(foreground_pixels)
kmeans_bg = KMeans(n_clusters=8, random_state=42).fit(background_pixels)

def compute_likelihood(pixel, kmeans):
    return np.min(np.linalg.norm(pixel - kmeans.cluster_centers_, axis=1))

height, width, _ = original_image.shape
segmentation_mask = np.zeros((height, width), dtype=np.uint8)

for i in range(height):
    for j in range(width):
        likelihood_fg = compute_likelihood(original_image[i, j], kmeans_fg)
        likelihood_bg = compute_likelihood(original_image[i, j], kmeans_bg)
        segmentation_mask[i, j] = 255 if likelihood_fg < likelihood_bg else 0

# Refine segmentation mask
segmentation_mask = cv2.morphologyEx(segmentation_mask, cv2.MORPH_CLOSE, kernel)

# Apply segmentation mask to original image
foreground_color = cv2.bitwise_and(original_image, original_image, mask=segmentation_mask)

background_black = np.zeros_like(original_image)

# Use the segmentation mask to isolate the background in the black image
# No need to invert the mask since we're directly using the foreground mask
background = cv2.bitwise_and(background_black, background_black, mask=cv2.bitwise_not(segmentation_mask))

# Combine the color foreground with the black background
combined_image = cv2.add(foreground_color, background)

# Save the colored foreground with black background image
output_colored_path = 'Lady-segmentation.png'
cv2.imwrite(output_colored_path, combined_image)

print(f"Segmentation with colored foreground and black background saved to {output_colored_path}")


<>:5: SyntaxWarning: invalid escape sequence '\l'
<>:6: SyntaxWarning: invalid escape sequence '\l'
<>:5: SyntaxWarning: invalid escape sequence '\l'
<>:6: SyntaxWarning: invalid escape sequence '\l'
C:\Users\Talal_HP\AppData\Local\Temp\ipykernel_29440\155565899.py:5: SyntaxWarning: invalid escape sequence '\l'
  original_image = cv2.imread('Q1 Dataset\lady.png')
C:\Users\Talal_HP\AppData\Local\Temp\ipykernel_29440\155565899.py:6: SyntaxWarning: invalid escape sequence '\l'
  aux_image = cv2.imread('Q1 Dataset\lady stroke 1.png')


Segmentation with colored foreground and black background saved to Lady-segmentation.png


## With Custom Kmean

In [40]:
import cv2
import numpy as np

def kmeans(data, k, n_iterations=100):
    np.random.seed(42)
    indices = np.random.choice(len(data), k, replace=False)
    centers = data[indices]
    
    for _ in range(n_iterations):
        distances = np.sqrt(((data - centers[:, np.newaxis])**2).sum(axis=2))
        closest_cluster = np.argmin(distances, axis=0)
        new_centers = np.array([data[closest_cluster == j].mean(axis=0) for j in range(k)])
        
        if np.all(centers == new_centers):
            break
        centers = new_centers
    
    return centers

def compute_likelihood(pixel, centers):
    return np.min(np.linalg.norm(pixel - centers, axis=1))

def generate_masks_from_strokes(stroke_image_paths):
    aggregated_foreground_mask = None
    aggregated_background_mask = None

    for path in stroke_image_paths:
        aux_image = cv2.imread(path)
        aux_hsv = cv2.cvtColor(aux_image, cv2.COLOR_BGR2HSV)

        foreground_mask = cv2.inRange(aux_hsv, red_lower, red_upper)
        background_mask = cv2.inRange(aux_hsv, blue_lower, blue_upper)

        kernel = np.ones((5, 5), np.uint8)
        foreground_mask = cv2.morphologyEx(foreground_mask, cv2.MORPH_CLOSE, kernel)
        background_mask = cv2.morphologyEx(background_mask, cv2.MORPH_CLOSE, kernel)

        if aggregated_foreground_mask is None:
            aggregated_foreground_mask = foreground_mask
        else:
            aggregated_foreground_mask = cv2.bitwise_or(aggregated_foreground_mask, foreground_mask)

        if aggregated_background_mask is None:
            aggregated_background_mask = background_mask
        else:
            aggregated_background_mask = cv2.bitwise_or(aggregated_background_mask, background_mask)

    return aggregated_foreground_mask, aggregated_background_mask

original_image = cv2.imread('Q1 Dataset/lady.png')

# List of paths to your stroke images
stroke_image_paths = ['Q1 Dataset/lady stroke 1.png', 'Q1 Dataset/lady stroke 2.png']  # Add your paths here

red_lower = np.array([0, 100, 50])
red_upper = np.array([10, 255, 255])
blue_lower = np.array([100, 100, 50])
blue_upper = np.array([130, 255, 255])

foreground_mask, background_mask = generate_masks_from_strokes(stroke_image_paths)

foreground_pixels = original_image[foreground_mask == 255]
background_pixels = original_image[background_mask == 255]

if foreground_pixels.size == 0 or background_pixels.size == 0:
    raise ValueError("No seeds detected. Please check the color thresholds and auxiliary image.")

centers_fg = kmeans(foreground_pixels.reshape(-1, 3), 3)
centers_bg = kmeans(background_pixels.reshape(-1, 3), 3)

height, width, _ = original_image.shape
segmentation_mask = np.zeros((height, width), dtype=np.uint8)

for i in range(height):
    for j in range(width):
        likelihood_fg = compute_likelihood(original_image[i, j], centers_fg)
        likelihood_bg = compute_likelihood(original_image[i, j], centers_bg)
        segmentation_mask[i, j] = 255 if likelihood_fg < likelihood_bg else 0

kernel = np.ones((5, 5), np.uint8)
segmentation_mask = cv2.morphologyEx(segmentation_mask, cv2.MORPH_CLOSE, kernel)

foreground_color = cv2.bitwise_and(original_image, original_image, mask=segmentation_mask)
background_black = np.zeros_like(original_image)
background = cv2.bitwise_and(background_black, background_black, mask=cv2.bitwise_not(segmentation_mask))
combined_image = cv2.add(foreground_color, background)

output_colored_path = 'Lady-segmentation.png'
cv2.imwrite(output_colored_path, combined_image)

print(f"Segmentation with colored foreground and black background saved to {output_colored_path}")


Segmentation with colored foreground and black background saved to Lady-segmentation.png
